# Table of Contents
1. Introduction
2. Loading and Viewing Data 
3. Data Summary and Questions
4. Pre-processing
5. Visualizations 
6. Conclusion

# 1. Introduction

This notebook explores a scraped craigslist dataset, `craigslist.csv`, and walks through my process for exploring, pre-processing, and extracting / visualizing insights from the data. 

The dataset was given by USF Data Science professor, Aleks Taranov. Minimal information was provided on this dataset other than the fact that it was scraped from craigslist. See the conclusion section for an overview insights I found interesting after exploration. 

### 1.1 Packages used

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks

init_notebook_mode(connected=True)

# 2. Loading and Viewing Data

In [2]:
data = pd.read_csv('data.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.head(2)

,title,text,latitude,longitude,city_text,date_posted,date_updated,price,deleted,sqft,bedrooms,bathrooms,pets,laundry,parking,craigslist,place,city,state,county
0,$895 / 1br - 500ft2 - 1 BD + 1 BD WATER PAID$ ...,QR Code Link to This Post\n \n ...,34.860788,-118.170104,W. ROSAMOND BLVD. / B ST.,2018-10-08 22:29:36,2018-10-15 13:07:37,895.0,False,500.0,1.0,1.0,none,shared,off-street,losangeles,Rosamond,NaN,CA,Kern
1,$895 / 1br - 500ft2 - 1 BD + 1 BD WATER PAID$ ...,QR Code Link to This Post\n \n ...,34.861017,-118.169932,W. ROSAMOND BLVD. / B ST.,2018-10-15 11:44:23,NaN,895.0,False,500.0,1.0,1.0,none,shared,off-street,losangeles,Rosamond,NaN,CA,Kern


### 2.1 Basic Stats

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21948 entries, 0 to 21947
Data columns (total 20 columns):
title           21947 non-null object
text            21947 non-null object
latitude        21864 non-null float64
longitude       21864 non-null float64
city_text       20287 non-null object
date_posted     21947 non-null object
date_updated    8809 non-null object
price           21845 non-null float64
deleted         21948 non-null bool
sqft            16357 non-null float64
bedrooms        20900 non-null float64
bathrooms       20900 non-null float64
pets            21655 non-null object
laundry         21732 non-null object
parking         21649 non-null object
craigslist      21948 non-null object
place           21247 non-null object
city            20092 non-null object
state           21853 non-null object
county          21853 non-null object
dtypes: bool(1), float64(6), object(13)
memory usage: 3.2+ MB


In [4]:
data.describe()

,latitude,longitude,price,sqft,bedrooms,bathrooms
count,21864.000000,21864.000000,2.184500e+04,16357.000000,20900.000000,20900.000000
mean,36.694257,-120.871645,4.488750e+03,885.330195,1.489809,1.326124
std,2.024755,2.327231,2.402130e+05,1588.511928,0.791082,0.520324
min,28.311200,-123.340577,0.000000e+00,1.000000,0.000000,0.000000
25%,34.208205,-122.299800,1.720000e+03,680.000000,1.000000,1.000000
50%,37.576840,-121.953605,2.288000e+03,835.000000,1.000000,1.000000
75%,37.915600,-118.472888,2.915000e+03,1025.000000,2.000000,2.000000
max,47.680645,-72.922161,3.408374e+07,200000.000000,7.000000,6.000000


**Notes from stats:**
   * odd stats for min price, sqft, bedrooms, and bathroom (0,1,0,0) respectively. 
       * studios are listed as having 0 bedrooms 
   * Missing values across multiple columns 
   * Most likely an outlier in price and sqft 
   

**Digging into outliers on high / low end:**

In [5]:
data.quantile([0.99,1])

,latitude,longitude,price,deleted,sqft,bedrooms,bathrooms
0.99,38.780016,-116.956917,6283.56,0.0,1817.0,3.0,2.5
1.00,47.680645,-72.922161,34083742.00,1.0,200000.0,7.0,6.0


In [6]:
data.quantile([0,.01])

,latitude,longitude,price,deleted,sqft,bedrooms,bathrooms
0.00,28.311200,-123.340577,0.0,0.0,1.00,0.0,0.0
0.01,32.623185,-122.742790,895.0,0.0,328.24,0.0,1.0


There are obvious outliers on both the high and low ends based on price and sqft. Let's go ahead and remove those

In [7]:
data = data[(data.price <= data.price.quantile(0.99)) \
            & (data.sqft <= data.sqft.quantile(0.99))]
data = data[(data.price >= data.price.quantile(0.01)) \
            & (data.sqft >= data.sqft.quantile(0.01))]

In [8]:
len(data)

15813

In [9]:
data.describe()

,latitude,longitude,price,sqft,bedrooms,bathrooms
count,15790.000000,15790.000000,15813.000000,15813.000000,15485.000000,15485.000000
mean,36.688837,-120.826029,2424.495542,862.193828,1.515725,1.357540
std,2.047431,2.294057,935.352852,248.797432,0.719933,0.504387
min,28.311200,-123.215663,950.000000,326.000000,0.000000,0.000000
25%,34.214762,-122.249472,1750.000000,685.000000,1.000000,1.000000
50%,37.529542,-121.915300,2290.000000,835.000000,2.000000,1.000000
75%,38.008321,-118.477679,2875.000000,1016.000000,2.000000,2.000000
max,47.680645,-72.922161,6269.000000,1817.000000,5.000000,6.000000


### 2.2 Unique Values and Counts 

Counties:

In [10]:
print(data.county.unique())
print(f'number of unique counties: {len(data.county.unique())}')

['Los Angeles' 'Kern' nan 'Orange' 'Ventura' 'San Bernardino'
 'San Luis Obispo' 'Sacramento' 'Solano' 'Yolo' 'Placer' 'El Dorado'
 'Marin' 'San Joaquin' 'Sonoma' 'Fairfield' 'Spokane' 'Santa Clara'
 'San Francisco' 'San Diego' 'Washoe' 'Alameda' 'Clark' 'Riverside'
 'Contra Costa' 'Napa' 'San Mateo' 'New Haven' 'Lake' 'Mendocino' 'Duval'
 'Virginia Beach' 'Franklin' 'Polk' 'Monterey' 'Montgomery']
number of unique counties: 36


Cities:

In [11]:
print(data.city.unique())
print(f'number of unique counties: {len(data.city.unique())}')

['Lancaster' 'Santa Clarita' nan 'Palmdale' 'Glendale' 'Los Angeles'
 'West Hollywood' 'La Verne' 'West Covina' 'Pasadena' 'Long Beach'
 'Alhambra' 'Inglewood' 'Santa Fe Springs' 'San Fernando' 'La Habra'
 'Bellflower' 'Downey' 'Burbank' 'Cerritos' 'Garden Grove' 'Norwalk'
 'Paramount' 'Carson' 'Whittier' 'Pico Rivera' 'La Mirada' 'La Palma'
 'Lynwood' 'Compton' 'Lakewood' 'Thousand Oaks' 'Culver City'
 'Simi Valley' 'Covina' 'La Canada Flintridge' 'Agoura Hills' 'Monrovia'
 'Azusa' 'Diamond Bar' 'Glendora' 'Baldwin Park' 'Montebello'
 'South Pasadena' 'San Gabriel' 'El Monte' 'Pomona' 'Arcadia' 'Rosemead'
 'Duarte' 'Monterey Park' 'Claremont' 'Santa Monica' 'Torrance'
 'Redondo Beach' 'Hermosa Beach' 'Hawthorne' 'Pismo Beach' 'Beverly Hills'
 'Lomita' 'Malibu' 'Rancho Palos Verdes' 'Gardena' 'Sacramento'
 'Vacaville' 'West Sacramento' 'Roseville' 'Citrus Heights' 'Folsom'
 'Rancho Cordova' 'Rocklin' 'Davis' 'Elk Grove' 'Woodland' 'Novato' 'Lodi'
 'Colfax' 'Petaluma' 'Suisun City' 'San

### 2.3 Count NAs 

In [12]:
sum(data.city.isna())

1398

### 2.4 date_posted:

Cleaning up `date_posted`

In [13]:
# convert date_posted to time stamp 
data.date_posted = pd.to_datetime(data.date_posted)
# drop all rows that have NA for timestamp 
data.drop(index=data[data.date_posted.isna()].index,inplace=True)
# reformate date_time to month day year 
data['date_posted'] = data.date_posted\
    .apply(lambda x: x.strftime('%m-%d-%Y'))

Date Range:

In [14]:
# grab dates and sort 
start_date = min(data.date_posted)
end_date = max(data.date_posted)
print(f'start date is {start_date}')
print(f'end date is {end_date}')

start date is 09-08-2018
end date is 10-15-2018


### 2.5 Num Observations

In [15]:
len(data)

15813

# 3. Data Summary and Questions

### 3.1 Summary: 
* 16022 Observations, originally 21947 observations, but I considered sqft to be necessary in understanding the data and dropped those rows where sqft were not included. 
* Date range 09-08-2018 to 10-15-2018
* There are 38 distinct unique counties. 
* There are 190 distinct unique cities. 
* Observations with a price of \$0 and sf of 1 were removed as outliers. 


### 3.2 Questions to explore:
* Can we discern between legitimate housing posts / realtor postings / homeowner postings?
    * If so, is there a relationship to any other feature? Potentially:
        * post length by word count 
        * housing price 
        * location 
        * post sentiment 
        * link / no link 
        * anti-spam score

# 5. Visualizations

Let's look at the distribution of price per squarefoot per county:

In [16]:
data['prc_sf'] = data.price/data.sqft

# create second dataframe for easier boxplot 
data2 = data[['prc_sf','county']]

# grab top 10 counties by records 
top10 = list(data2.county.value_counts()[:10].index)
data2 = data2[data2.county.isin(top10)]

# pivot dataframe and sort by price per squarefoot 
data_piv = data2.pivot(columns='county', values='prc_sf')
keys_sorted = list(data2.groupby(by='county').mean().sort_values(by='prc_sf',ascending=False).index)

# plot boxplots 
data_piv.iplot(
        kind='box',
        keys=keys_sorted,
        layout=dict(
            height=600,
            yaxis=dict(title='Price per Square Foot'),
#             title='',
            margin=dict(b=140)))

/Users/benjaminkhuong/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Based on the boxplot above, it would be interesting to explore **San Francisco, Los Angeles, and Sacramento** as they have fairly different price per squarefoot distributions. 

In [17]:
# subset LA, SF, and SAC 
selected_counties = data[['prc_sf', 'county',
                          'sqft', 'price', 'bedrooms', 'bathrooms']]
selected_counties.dropna(inplace=True)
selected_counties = selected_counties[selected_counties.county.isin(
    ['Los Angeles', 'San Francisco', 'Sacramento'])]

# create variable bedbath
selected_counties['bedbath'] = selected_counties['bedrooms'] + selected_counties['bathrooms']

# increase difference in magnitude for bubble sizes 
selected_counties['bedbath'] = selected_counties['bedbath']*2.5

In [18]:
data_scatter2 = selected_counties[selected_counties.county=='San Francisco']
text2 = []
for i in range(len(data_scatter2)):
    text2.append(f'<b>San Francisco</b>\
                 <br><b>Sqft</b>: {int(data_scatter2.sqft.iloc[i])}\
                 <br><b>Price:</b> {int(data_scatter2.price.iloc[i])}\
                 <br><b>Bedbaths: </b>{int(data_scatter2.bedbath.iloc[i])}')

trace2 = go.Scatter(
    x = data_scatter2.sqft,
    y = data_scatter2.price,
    name='San Francisco',
    mode='markers',
    text = text2,
    hoverinfo='text',
    marker=dict(
        color ='navy', #set color equal to a variable
        opacity=1,
        size=list(data_scatter2.bedbath.values)
    )
)

data_scatter1 = selected_counties[selected_counties.county=='Los Angeles']
text1 = []
for i in range(len(data_scatter1)):
    text1.append(f'<b>Los Angeles</b>\
                 <br><b>Sqft</b>: {int(data_scatter1.sqft.iloc[i])}\
                 <br><b>Price:</b> {int(data_scatter1.price.iloc[i])}\
                 <br><b>Bedbaths: </b>{int(data_scatter1.bedbath.iloc[i])}')

trace1 = go.Scatter(
    x = data_scatter1.sqft,
    y = data_scatter1.price,
    name='Los Angeles',
    mode='markers',
    hoverinfo='text',
    text = text1,
    marker=dict(
        color = 'rgb(79, 146, 255)', #set color equal to a variable
        opacity=1,
        size=list(data_scatter1.bedbath.values)
    )
)

data_scatter3 = selected_counties[selected_counties.county=='Sacramento']
text3 = []
for i in range(len(data_scatter3)):
    text3.append(f'<b>Sacramento </b> <br><b>Sqft</b>: {int(data_scatter3.sqft.iloc[i])}\
                 <br><b>Price:</b> {int(data_scatter3.price.iloc[i])}\
                 <br><b>Bedbaths: </b>{int(data_scatter3.bedbath.iloc[i])}')

trace3 = go.Scatter(
    x = data_scatter3.sqft,
    y = data_scatter3.price,
    text = text3,
    mode='markers',
    name = 'Sacramento',
    hoverinfo='text',
    marker=dict(
        color = 'rgb(198, 220, 255)', #set color equal to a variable
        opacity=1,
        size=list(data_scatter3.bedbath.values)
    )
)

layout = dict(title = 'Price vs Square Feet for SF, LA, and SAC',
              xaxis = dict(title = 'Square Feet'),
              yaxis = dict(title = 'Price (in dollars)'),
              )

data = [trace2, trace1, trace3]
fig = dict(data=data, layout=layout)
iplot(fig, filename='bubblechart-color')